### Building the Playlist Creation Model

In [1]:
#Imports necessary notebooks
import spotipy
from spotipy import util
import numpy as np
import pandas as pd
import time
import matplotlib
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
import statsmodels.api as sm
from statsmodels.api import OLS
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import LassoCV
import sklearn.metrics as metrics
from sklearn.model_selection import cross_val_score
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LinearRegression
%matplotlib inline

import seaborn as sns

/Users/sarah/anaconda/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


First, we read in the data that was called from the Spotify API. This dataset includes 668 unique playlists from various Spotify-featured genres. For each playlist, there is playlist-specific data (followers, length, etc.) as well as aggregate data about its tracks (average danceability, tempo, etc.)

In [2]:
playlists = pd.read_csv('playlists.csv')
playlists.head()

,Unnamed: 0,Unnamed: 0.1,index,collaborative,external_urls,href,id,images,name,owner,...,explicit,popularity,acousticness,danceability,energy,liveness,loudness,speechiness,tempo,valence
0,0,0,0,False,{'spotify': 'https://open.spotify.com/user/spo...,https://api.spotify.com/v1/users/spotify/playl...,37i9dQZF1DXcBWIGoYBM5M,"[{'height': 300, 'url': 'https://i.scdn.co/ima...",Today's Top Hits,spotify,...,0.368421,83.631579,0.195290,0.670779,0.672232,0.170601,-5.296589,0.089700,119.590074,0.428257
1,2,1,1,False,{'spotify': 'https://open.spotify.com/user/spo...,https://api.spotify.com/v1/users/spotify/playl...,37i9dQZF1DX0XUsuxWHRQd,"[{'height': 300, 'url': 'https://i.scdn.co/ima...",RapCaviar,spotify,...,0.989130,77.173913,0.166482,0.757576,0.629641,0.198623,-6.440326,0.221160,130.062750,0.430804
2,4,2,2,False,{'spotify': 'https://open.spotify.com/user/spo...,https://api.spotify.com/v1/users/spotify/playl...,37i9dQZF1DX4dyzvuaRJ0n,"[{'height': 300, 'url': 'https://i.scdn.co/ima...",mint,spotify,...,0.094737,65.357895,0.116051,0.604547,0.769189,0.186685,-5.135021,0.067026,125.485221,0.390440
3,6,3,3,False,{'spotify': 'https://open.spotify.com/user/spo...,https://api.spotify.com/v1/users/spotify/playl...,37i9dQZF1DXcF6B6QPhFDv,"[{'height': 300, 'url': 'https://i.scdn.co/ima...",Rock This,spotify,...,0.125000,60.267857,0.052656,0.532304,0.785554,0.207500,-5.404982,0.062518,124.610000,0.519429
4,7,4,4,False,{'spotify': 'https://open.spotify.com/user/spo...,https://api.spotify.com/v1/users/spotify/playl...,37i9dQZF1DX4SBhb3fqCJd,"[{'height': 300, 'url': 'https://i.scdn.co/ima...",Are & Be,spotify,...,0.500000,64.558824,0.224200,0.629294,0.549559,0.160476,-7.193765,0.127618,113.833868,0.458021


### Feature Selection

Given that we have many potential predictors to work with (see the list below), we performed feature selection through exhaustive search selection. We perform feature selection within each playlist genre, given that the factors that will make a playlist successful intuitively differ between genres. We also attempted stepwise forward/backward selection, but the sample size of playlists per genre was too small to achieve satisfactory results. 

In [3]:
#List of columns in the initial dataset 
list(playlists)

['Unnamed: 0',
 'Unnamed: 0.1',
 'index',
 'collaborative',
 'external_urls',
 'href',
 'id',
 'images',
 'name',
 'owner',
 'public',
 'snapshot_id',
 'tracks',
 'type',
 'uri',
 'Category',
 'followers',
 'length',
 'avgpopularity',
 'medianpopularity',
 'pid',
 'duration_ms',
 'explicit',
 'popularity',
 'acousticness',
 'danceability',
 'energy',
 'liveness',
 'loudness',
 'speechiness',
 'tempo',
 'valence']

In [4]:
#Exhaustive search selection
import itertools

def exhaustive_search_selection(x, y):
    """Exhaustively search predictor combinations. .

    Parameters:
    -----------
    x : DataFrame of predictors/features
    y : response varible 
    
    
    Returns:
    -----------
    
    Dataframe of model comparisons and OLS Model with 
    lowest BIC for subset with highest R^2
    
    """
    
    # total no. of predictors
    d = x.shape[1]
    predictors = x.columns
    overall_min_bic = 10000 # A big number 
    output = dict()
    
    # Outer loop: iterate over sizes 1 .... d
    for k in range(1,d):
        
        max_r_squared = -10000 # A small number
        
        # Enumerate subsets of size ‘k’
        subsets_k = itertools.combinations(predictors, k)
        
        # Inner loop: iterate through subsets_k
        for subset in subsets_k:
            # Fit regression model using ‘subset’ and calculate R^2 
            # Keep track of subset with highest R^2
            
            features = list(subset)
            x_subset = x[features]
            
            model = OLS(y, x_subset)
            results = model.fit()
            r_squared = results.rsquared
            
            # Check if we get a higher R^2 value than than current max R^2, 
            # if so, update our best subset 
            if(r_squared > max_r_squared):
                max_r_squared = r_squared
                best_subset = features
                best_model = model
                best_formula = "y ~ {}".format(' + '.join(features))
        
        results = best_model.fit()
        bic = results.bic
        if bic < overall_min_bic:
            overall_min_bic = bic 
            best_overall_subset = best_subset
            best_overall_rsquared = results.rsquared
            best_overall_formula = best_formula
            best_overall_model = best_model
        
        #print("For k={0} the best model is {1} with bic={2:.2f} and R^2={3:.4f}".format(k,best_formula,bic,results.rsquared))
        output[k] = {'best_model':best_formula, 'bic':bic,'r_squared':results.rsquared}
        
    #print("The best overall model is {0} with bic={1:.2f} and R^2={2:.3f}".format(best_overall_formula,overall_min_bic, best_overall_rsquared))
    
    return pd.DataFrame(output).T,best_overall_model,best_overall_subset

Before proceeding with feature selection, we filter out the irrelevant variables. These include variables containing information like playlist ID, or variables that cannot be aggregated into a model.

In [5]:
#Filtered columns of interest
cols = ['followers',
 'duration_ms',
 'popularity',
 'acousticness',
 'danceability',
 'energy',
 'liveness',
 'loudness',
 'speechiness',
 'tempo',
 'valence']

In [6]:
#Preparing a DataFrame
dfcols = ('BIC','R2','Features')
catmodels = pd.DataFrame(index=playlists['Category'].unique(),columns=dfcols)

Now we conduct the feature selection, along with an initial OLS model based on the selected features. The selected features, BIC and R2 of each model is returned per genre, in the DataFrame below.

In [7]:
#Performing Exhaustive Search Selection; Results are Below
for cat in playlists['Category'].unique():
    g1 = playlists.loc[playlists['Category'] == cat]
    g = g1[cols]
    x = g.drop(['followers'],axis=1)
    y = g['followers']
    stats,model,features = exhaustive_search_selection(x,y)
    bestx = g[features]
    model = OLS(y, bestx)
    results = model.fit()
    bic = results.bic
    r_squared = results.rsquared
    catmodels.at[cat,'BIC'] = bic
    catmodels.at[cat,'R2'] = r_squared
    catmodels.at[cat,'Features'] = features
catmodels

,BIC,R2,Features
toplists,436.215,0.700129,"[popularity, tempo]"
chill,1166.46,0.604576,"[popularity, loudness]"
mood,891.997,0.435306,[popularity]
pop,373.588,0.924562,"[acousticness, danceability, energy, liveness,..."
edm_dance,1154.91,0.693318,"[popularity, danceability]"
hiphop,570.006,0.440007,[popularity]
party,378.419,0.913579,"[popularity, danceability, energy, loudness, t..."
rock,729.646,0.531455,"[popularity, liveness]"
workout,746.553,0.66118,"[duration_ms, popularity]"
focus,419.172,0.877931,"[duration_ms, popularity, energy, liveness, lo..."


## Model Selection

In addition to the OLS model, we explored other options for building models that predict success for a playlist. Below, we added polynomial values to each model to see whether a polynomial OLS model would perform better than a linear model. However, based on BIC, the linear model consistently performed better; we ultimately did choose the linear model.

We also experimented with other types of classification models, such as decision trees and random forests (sample code at the bottom). However, the sample size per genre was ultimately too small to generate robust results. More importantly, such models do not provide interpretable coefficients. We needed to have coefficients/a regression equation to be able to conduct the next step, generating a successful playlist based on the model. All of these factors strengthened the argument for selecting the linear model.

In [8]:
for cat in playlists['Category'].unique():
    #Data Setup
    g1 = playlists.loc[playlists['Category'] == cat]
    g = g1[cols]
    feats = catmodels.loc[cat,'Features']
    train, test = train_test_split(g, test_size=0.5, random_state=1000)
    y_train = train['followers']
    x_train = train[feats]
    y_test = test['followers']
    x_test = test[feats]
    
    degree = 2
    poly = PolynomialFeatures(degree)

    x_train_poly = poly.fit_transform(x_train)
    x_test_poly = poly.fit_transform(x_test)
    model = OLS(y_train, x_train_poly)
    results = model.fit()
    bic = results.bic
    r_squared = results.rsquared
    catmodels.at[cat,'Poly BIC'] = bic
    catmodels.at[cat,'Poly R2'] = r_squared
catmodels

/Users/sarah/anaconda/lib/python3.6/site-packages/statsmodels/regression/linear_model.py:1386: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss


,BIC,R2,Features,Poly BIC,Poly R2
toplists,436.215,0.700129,"[popularity, tempo]",-88.199672,1.000000
chill,1166.46,0.604576,"[popularity, loudness]",586.239176,0.723811
mood,891.997,0.435306,[popularity],431.798758,0.747629
pop,373.588,0.924562,"[acousticness, danceability, energy, liveness,...",-192.611815,1.000000
edm_dance,1154.91,0.693318,"[popularity, danceability]",575.743415,0.724887
hiphop,570.006,0.440007,[popularity],264.889781,0.342404
party,378.419,0.913579,"[popularity, danceability, energy, loudness, t...",-160.868050,1.000000
rock,729.646,0.531455,"[popularity, liveness]",355.173060,0.771883
workout,746.553,0.66118,"[duration_ms, popularity]",366.290231,0.608226
focus,419.172,0.877931,"[duration_ms, popularity, energy, liveness, lo...",-168.357843,1.000000


In [9]:
#Data Manipulation: Combining Playlist & Track Data
fulltrackfeatures = pd.read_csv('allfeaturesfinal.csv')
playlistcats = playlists[['pid','Category']]
fulltrackfeatures = fulltrackfeatures.merge(playlistcats,on='pid')

## Generating the Playlists

After selecting the model and features for each genre, we created the below function that returns a playlist of 30 songs. The function takes in one argument: a requested playlist genre. The function first subsets our full list of tracks, only selecting tracks that appeared on at least one playlist of the requested genre. Then, it subsets the predictor variables based on the features selected in the previous step. Next, the predicted value of each track is calculated according to the linear model for the genre. The 30 top-performing tracks are selected and returned in a DataFrame, along with the predicted number of followers that the playlist will receive.

The function also compares the generated playlist to a baseline model, that selects tracks solely based on those that are the most popular. Intuitively, this is a strong baseline model because of the high correlation between track popularity and number of playlist followers discussed in the EDA section. The number of predicted followers for the baseline model is also returned in the function. 

Below, there is a sample output for one call of the function, as well as a comparison of the model's performance with the baseline.

In [10]:
#List of potential playlist genres 

catlist = list(playlists['Category'].unique())
catlist

['toplists',
 'chill',
 'mood',
 'pop',
 'edm_dance',
 'hiphop',
 'party',
 'rock',
 'workout',
 'focus',
 'decades',
 'dinner',
 'sleep',
 'indie_alt',
 'rnb',
 'popculture',
 'metal',
 'soul',
 'romance',
 'jazz',
 'classical',
 'latin',
 'country',
 'folk_americana',
 'blues',
 'travel',
 'kids',
 'reggae',
 'gaming',
 'punk',
 'funk',
 'comedy']

In [11]:
def create_playlists(genre):
    tracksubset = fulltrackfeatures.loc[fulltrackfeatures['Category'] == genre]
    tracksubset = tracksubset.drop_duplicates('id')
    feats = catmodels.loc[cat,'Features']
    x = tracksubset[feats]
    g1 = playlists.loc[playlists['Category'] == genre]
    g = g1[cols]
    y = g['followers']
    bestx = g[feats]
    bestx = sm.add_constant(bestx)
    model = OLS(y, bestx)
    results = model.fit()
    params = results.params
    for index,row in x.iterrows():
        coeffs = list()
        for f in range(len(feats)):
            pred = row[f]
            param = params[f+1]
            co = pred*param
            coeffs.append(co)
        tracksubset.at[index,'Followers'] = np.sum(coeffs) + params[0]
    playlist = tracksubset.nlargest(30, 'Followers', keep='first')
    baseline = tracksubset.nlargest(30, 'popularity', keep='first')
    means = playlist.mean(axis=0)
    basemeans = baseline.mean(axis=0)
    predict_x = means[feats]
    predict_x = predict_x.values
    predict_x = np.insert(predict_x,0,1.)    
    predict_baseline = basemeans[feats]
    predict_baseline = predict_baseline.values
    predict_baseline = np.insert(predict_baseline,0,1.)    
    playlistfollowers = results.predict(predict_x)
    baselinefollowers = results.predict(predict_baseline)
    return playlist, playlistfollowers, baselinefollowers

In [13]:
#Comparing Results to Baseline
comparison = pd.DataFrame(index=catlist, columns = ['Playlist Followers', 'Baseline Followers'])

In [15]:
for c in catlist:
    playlist, followers, baseline = create_playlists(c)
    comparison.at[c,'Playlist Followers'] = followers
    comparison.at[c, 'Baseline Followers'] = baseline

comparison['Difference'] = comparison['Playlist Followers'] - comparison['Baseline Followers']
comparison

,Playlist Followers,Baseline Followers,Difference
toplists,[17243764.7281],[9295405.23792],[7948359.49018]
chill,[1934683.16227],[1627697.47106],[306985.691209]
mood,[2388926.70725],[1926176.47865],[462750.228599]
pop,[1623408.3223],[1438877.03593],[184531.286371]
edm_dance,[1588029.70216],[1568294.47887],[19735.2232887]
hiphop,[2139579.02761],[1945807.2438],[193771.783808]
party,[3497045.27714],[2302931.27116],[1194114.00598]
rock,[2502834.98699],[1111357.44394],[1391477.54305]
workout,[2130917.79577],[2047996.13502],[82921.6607459]
focus,[2963728.79173],[1673828.81446],[1289899.97727]


The highly positive values in the "comparison" column show that our model was successful in generating good playlists for each genre.

In [ ]:
#Decision tree models, using train/test split, that didn't work:

for cat in playlists['Category'].unique():
    #Data Setup
    g1 = playlists.loc[playlists['Category'] == cat]
    g = g1[cols]
    feats = catmodels.loc[cat,'Features']
    train, test = train_test_split(g, test_size=0.5, random_state=1000)
    y_train = train['followers']
    x_train = train[feats]
    y_test = test['followers']
    x_test = test[feats]
    
    #OLS
    model = OLS(y_train, x_train)
    results = model.fit()
    predictions = results.predict(x_test)
    score = np.mean(predictions==y_test)
    catmodels.at[cat,'OLS Score'] = score

    #Decision Tree
    depth = []
    tree_start = 3
    tree_end   = 20
    for i in range(tree_start,tree_end):
        dt = DecisionTreeClassifier(max_depth=i)
        scores = metrics.accuracy_score(y_test, dt.fit(x_train, y_train).predict(x_test))
        depth.append((i,scores.mean()))
    best_depth = np.argmax(np.array(depth)[:,1]) + tree_start
    dt = DecisionTreeClassifier(max_depth=best_depth)
    dt_fitted = dt.fit(x_train, y_train)
    dtscore = dt_fitted.score(x_test, y_test)
    catmodels.at[cat,'Tree Score'] = dtscore
    
    #Random Forest
    trees = [2**x for x in range(7)]
    ntrees = dict()
    for n_trees in trees:
        rf = RandomForestClassifier(n_estimators=n_trees, max_features='sqrt')
        rfscore = metrics.accuracy_score(y_test, rf.fit(x_train, y_train).predict(x_test))
        ntrees[n_trees] = rfscore
    best_n = max(ntrees, key=ntrees.get)
    catmodels.at[cat,'RF Score'] = ntrees[best_n]
    